# Polymer simulation with dynamic spreading of chromatin state



In [3]:
# import some libaries
import sys
import os
import numpy as np
import numpy.matlib
import h5py
import ast
import pandas as pd
import math
import copy
import polychrom
from polychrom.starting_conformations import grow_cubic
from polychrom.hdf5_format import HDF5Reporter, list_URIs, load_URI, load_hdf5_file
from polychrom.simulation import Simulation
from polychrom import polymerutils
from polychrom import forces
from polychrom import forcekits
import time
import scipy
from scipy import spatial  

In [4]:
saveRoot = r'T:/2022-05-09_SedonaPcSims/'  # update save path
if not os.path.exists(saveRoot):
    os.mkdir(saveRoot)
saveFolder = saveRoot +'testSim/'
if not os.path.exists(saveFolder):
    os.mkdir(saveFolder)  # only creates folders 1 deep, won't create a full path

In [5]:
# add filepath to the Simulations folder inside the polychrom repository here:
sys.path.append(r'C:\Users\Alistair\Desktop\code\minrylab-polychrom\Simulations') 
from LEBondUpdater import bondUpdater 

In [7]:
# general parameters 

# color dynamic pars
iters = 2000 # number of iterations to do
totPc = 1/3 # Pc is 1/3 of the domain
onRate = 0.15 # .4 
onRateBkd = 0.0015;
offRate = 0.1 # .15 .5
contactRadius = 2 #2   3
nStates = 3
startLevel = 2

# perturbation
tStart = 0  # time to start pertubation
x1P = 0      # left coordinate of region to get ectopic Pc (state 3)
x2P = x1P + 0 # right coordinate of " " " "
x1R = 0  # left coordinate of region to lose Pc
x2R = x1R+0 # right coordinate of " " " "

density =  0.2  # density of the PBC box  (0.5 maybe a little high?  Helps in spreading though)
#  ==========Extrusion sim parameters====================
# there is probably a more elegant way to read in text values than ast.literal_eval, but this works.  
N1 = 600 #  Number of monomers in the polymer
M = 1  # separate chains in the same volume (will interact in trans with sticky sims)
N = N1 * M # number of monomers in the full simulation 

# compartment labels
oneChainMonomerTypes = np.zeros(N1).astype(int)
oneChainMonomerTypes[200:400] = 1 # mod self interaction
# create interaction matrix
interactionMatrix = np.array([[0, 0, 0], [0, 0.40, 0], [0, 0, 0]])  #=== KEY parameter  ===#
# ==== coil globule transition occurs around .7 for monomers of this length

# load prob
loadProb = np.ones([1,N1])  # uniform loading probability
loadProb = numpy.matlib.repmat(loadProb,1,M) # need to replicate and renormalize
loadProb = loadProb/np.sum(loadProb) 

# less common parameters
# Cohesin loop extrusion (simulations set to run without extrusion)
LIFETIME =  50 #  extruder lifetime
SEPARATION = 1e5 # simulations here with no extrusion (separation > monomer chain length)
ctcfSites =  np.array([100,200,300,400,500]) #CTCF site locations  (has no effect if no cohesin)
nCTCF = np.shape(ctcfSites)[0]
ctcfDir = np.zeros(nCTCF) # 0 is bidirectional, 1 is right 2 is left
ctcfCapture = 0.98*np.ones(nCTCF) #  capture probability per block if capture < than this, capture  
ctcfRelease =0.003*np.ones(nCTCF)  # % release probability per block. if capture < than this, release
lefPosFile = saveFolder + "LEFPos.h5"
LEFNum =  math.floor(N // SEPARATION )  # make 0 for no LEFs
monomers = N1 # 
attraction_radius = 1.5  #  try making this larger
num_chains = M  # simulation uses some equivalent chains  (5 in a real sim)
MDstepsPerCohesinStep = 800
smcBondWiggleDist = 0.2
smcBondDist = 0.5

# save pars
saveEveryBlocks = 10   # save every 10 blocks 
restartSimulationEveryBlocks = 100  # blocks per iteration
trajectoryLength =  iters*restartSimulationEveryBlocks #  1000 # time duration of simulation (down from 100,000)


# check that these loaded alright
print(f'LEF count: {LEFNum}')
print('interaction matrix:')
print(interactionMatrix)
print('monomer types:')
print(oneChainMonomerTypes)
print(saveFolder)




LEF count: 0
interaction matrix:
[[0.  0.  0. ]
 [0.  0.4 0. ]
 [0.  0.  0. ]]
monomer types:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0

In [8]:

#==================================#
# Run and load 1D simulation
#=================================#
import polychrom.lib.extrusion1Dv2 as ex1D # 1D classes 
ctcfLeftRelease = {}
ctcfRightRelease = {}
ctcfLeftCapture = {}
ctcfRightCapture = {}

# should modify this to allow directionality
for i in range(M): # loop over chains (this variable needs a better name Max)
    for t in range(len(ctcfSites)):
        pos = i * N1 + ctcfSites[t] 
        if ctcfDir[t] == 0:
            ctcfLeftCapture[pos] = ctcfCapture[t]  # if random [0,1] is less than this, capture
            ctcfLeftRelease[pos] = ctcfRelease[t]  # if random [0,1] is less than this, release
            ctcfRightCapture[pos] = ctcfCapture[t]
            ctcfRightRelease[pos] = ctcfRelease[t]
        elif ctcfDir[t] == 1: # stop Cohesin moving toward the right  
            ctcfLeftCapture[pos] = 0  
            ctcfLeftRelease[pos] = 1  
            ctcfRightCapture[pos] = ctcfCapture[t]
            ctcfRightRelease[pos] = ctcfRelease[t]
        elif ctcfDir[t] == 2:
            ctcfLeftCapture[pos] = ctcfCapture[t]  # if random [0,1] is less than this, capture
            ctcfLeftRelease[pos] = ctcfRelease[t]  # if random [0,1] is less than this, release
            ctcfRightCapture[pos] = 0
            ctcfRightRelease[pos] = 1
       
args = {}
args["ctcfRelease"] = {-1:ctcfLeftRelease, 1:ctcfRightRelease}
args["ctcfCapture"] = {-1:ctcfLeftCapture, 1:ctcfRightCapture}        
args["N"] = N 
args["LIFETIME"] = LIFETIME
args["LIFETIME_STALLED"] = LIFETIME  # no change in lifetime when stalled 

occupied = np.zeros(N)
occupied[0] = 1  # (I think this is just prevent the cohesin loading at the end by making it already occupied)
occupied[-1] = 1 # [-1] is "python" for end
cohesins = []

print('starting simulation with N LEFs=')
print(LEFNum)
for i in range(LEFNum):
    ex1D.loadOneFromDist(cohesins,occupied, args,loadProb) # load the cohesins 


with h5py.File(lefPosFile, mode='w') as myfile:
    
    dset = myfile.create_dataset("positions", 
                                 shape=(trajectoryLength, LEFNum, 2), 
                                 dtype=np.int32, 
                                 compression="gzip")
    steps = 100    # saving in 50 chunks because the whole trajectory may be large 
    bins = np.linspace(0, trajectoryLength, steps, dtype=int) # chunks boundaries 
    for st,end in zip(bins[:-1], bins[1:]):
        cur = []
        for i in range(st, end):
            ex1D.translocate(cohesins, occupied, args,loadProb)  # actual step of LEF dynamics 
            positions = [(cohesin.left.pos, cohesin.right.pos) for cohesin in cohesins]
            cur.append(positions)  # appending current positions to an array 
        cur = np.array(cur)  # when we finished a block of positions, save it to HDF5 
        dset[st:end] = cur
    myfile.attrs["N"] = N
    myfile.attrs["LEFNum"] = LEFNum
    
#=========== Load LEF simulation ===========#
trajectory_file = h5py.File(lefPosFile, mode='r')
LEFNum = trajectory_file.attrs["LEFNum"]  # number of LEFs
LEFpositions = trajectory_file["positions"]  # array of LEF positions  
steps = MDstepsPerCohesinStep # MD steps per step of cohesin  (set to ~800 in real sims)
Nframes = LEFpositions.shape[0] # length of the saved trajectory (>25000 in real sims)
print(f'Length of the saved trajectory: {Nframes}')
block = 0  # starting block 

# test some properties 
# assertions for easy managing code below 
assert (Nframes % restartSimulationEveryBlocks) == 0 
assert (restartSimulationEveryBlocks % saveEveryBlocks) == 0

savesPerSim = restartSimulationEveryBlocks // saveEveryBlocks
simInitsTotal  = (Nframes) // restartSimulationEveryBlocks
# concatinate monomers if needed
if len(oneChainMonomerTypes) != N:
    monomerTypes = np.tile(oneChainMonomerTypes, num_chains)
else:
    monomerTypes = oneChainMonomerTypes
    
N_chain = len(oneChainMonomerTypes)  
N = len(monomerTypes)
print(f'N_chain: {N_chain}')  # ~8000 in a real sim
print(f'N: {N}')   # ~40000 in a real sim
N_traj = trajectory_file.attrs["N"]
print(f'N_traj: {N_traj}')
assert N == trajectory_file.attrs["N"]
print(f'Nframes: {Nframes}')
print(f'simInitsTotal: {simInitsTotal}')

starting simulation with N LEFs=
0
Length of the saved trajectory: 200000
N_chain: 600
N: 600
N_traj: 600
Nframes: 200000
simInitsTotal: 2000


In [6]:
#==============================================================#
#                  RUN 3D simulation                              #
#==============================================================#



# Initial simulation using fixed input states
t=0
LEFsubset = LEFpositions[t*restartSimulationEveryBlocks:(t+1)*restartSimulationEveryBlocks,:,:] # a subset of the total LEF simulation time
milker = bondUpdater(LEFsubset)
data = grow_cubic(N,int((N/(density*1.2))**0.333))  # starting conformation
PBC_width = (N/density)**0.333
chains = [(N_chain*(k),N_chain*(k+1),0) for k in range(num_chains)]  # subchains in rpt
newFolder = saveFolder+'t'+str(0)+'/'
os.mkdir(newFolder)
reporter = HDF5Reporter(folder=newFolder, max_data_length=100)
a = Simulation(N=N, 
               error_tol=0.01, 
               collision_rate=0.02, 
               integrator ="variableLangevin", 
               platform="cuda",
               GPU = "0", 
               PBCbox=(PBC_width, PBC_width, PBC_width),
               reporters=[reporter],
               precision="mixed")  # platform="CPU", 
a.set_data(data) # initial polymer 
a.add_force(
    polychrom.forcekits.polymer_chains(
        a,
        chains=chains,
        nonbonded_force_func=polychrom.forces.heteropolymer_SSW,
        nonbonded_force_kwargs={
            'attractionEnergy': 0,  # base attraction energy for all monomers
            'attractionRadius': attraction_radius,
            'interactionMatrix': interactionMatrix,
            'monomerTypes': monomerTypes,
            'extraHardParticlesIdxs': []
        },
        bond_force_kwargs={
            'bondLength': 1,
            'bondWiggleDistance': 0.05
        },
        angle_force_kwargs={
            'k': 1.5
        }
    )
)
# ------------ initializing milker; adding bonds ---------
kbond = a.kbondScalingFactor / (smcBondWiggleDist ** 2)
bondDist = smcBondDist * a.length_scale
activeParams = {"length":bondDist,"k":kbond}
inactiveParams = {"length":bondDist, "k":0}
milker.setParams(activeParams, inactiveParams)  
milker.setup(bondForce=a.force_dict['harmonic_bonds'],
            blocks=restartSimulationEveryBlocks)
# If your simulation does not start, consider using energy minimization below
a.local_energy_minimization()  # only do this at the beginning

# this runs 
for i in range(restartSimulationEveryBlocks):   # loops over 100     
    if i % saveEveryBlocks == (saveEveryBlocks - 1):  
        a.do_block(steps=steps)
    else:
        a.integrator.step(steps)  # do steps without getting the positions from the GPU (faster)
    if i < restartSimulationEveryBlocks - 1: 
        curBonds, pastBonds = milker.step(a.context)  # this updates bonds. You can do something with bonds here
data = a.get_data()  # save data and step, and delete the simulation
del a    
reporter.blocks_only = True  # Write output hdf5-files only for blocks
time.sleep(0.2)  # wait 200ms for sanity (to let garbage collector do its magic)
reporter.dump_data()

# -------------------------------end of initialization. ---------------------------
# ===============   Time to start updating the color states  ================================#  
colorStates =  np.tile(startLevel*monomerTypes, [iters+1,1]) # initialize a matrix to store color states in. 
for t in range(iters):
    print(t)
    #==================== simulate epigenetic spreading 
    # load polymer
    # files = list_URIs(saveFolder)
    # data = load_URI(files[-1])  # this is the full data structure, it is possible we only want data['pos']

    newFolder = saveFolder+'t'+str(t+1)+'/'
    if not os.path.exists(newFolder):
        os.makedirs(newFolder)
    reporter = HDF5Reporter(folder=newFolder, max_data_length=100)
    print('creating folder')

    for p in range(M):
        polyDat = data[p*monomers:(p+1)*monomers,:]  # ['pos'][p*monomers:(p+1)*monomers,:]
        newColors = copy.copy(colorStates[t,p*monomers:(p+1)*monomers]) # note this is not a copy, just a reference. updating newColors immideately updates colorStates
        # moved frac bound down
        isLoss = np.random.rand(monomers) < offRate
        newColors[isLoss] = newColors[isLoss]-1 # was 0  # note, this immideately updates colorStates
        newColors[newColors<0] = 0
        fracBound = sum(newColors)/monomers/nStates
        fracFree = max(0,totPc-fracBound)
        ordr = np.random.permutation(monomers)
        dMap = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(polyDat))
        isOn = newColors # newColors == 1
        for o in ordr:
            isClose = dMap[o,:] < contactRadius
            tries = sum(isClose * isOn)  
            updateProb = onRate*tries*fracFree
            updateColor = np.random.rand() < updateProb  or  (np.random.rand(1) < onRateBkd*fracFree)  # ADDED FOR background integration
            if updateColor and newColors[o]<nStates:
                newColors[o] = newColors[o] + 1  # note, this immideately updates colorStates
                # colorStates[t+1,p*monomers+o] = 1     
                
            if t==tStart  and  o>x1P and o<x2P:  # perturbation at t=200
                newColors[o] = 3
            if t==tStart and  o>x1R and o<x2R:  # 
                newColors[o] = 0
                
        colorStates[t+1,p*monomers:(p+1)*monomers] = newColors
    # ============ run new polymer sim  ==========================    
    
    isPc =  colorStates[t+1,:]>0
    isPc = isPc.astype(int)
    LEFsubset = LEFpositions[t*restartSimulationEveryBlocks:(t+1)*restartSimulationEveryBlocks,:,:]
    milker = bondUpdater(LEFsubset)
    a = Simulation(N=N, 
                   error_tol=0.01, 
                   collision_rate=0.02, 
                   integrator ="variableLangevin", 
                   platform="cuda",
                   GPU = "0", 
                   PBCbox=(PBC_width, PBC_width, PBC_width),
                   reporters=[reporter],
                   precision="mixed")  # platform="CPU", 
    a.set_data(data) # initial polymer 
    a.add_force(
        polychrom.forcekits.polymer_chains(
            a,
            chains=chains,
            nonbonded_force_func=polychrom.forces.heteropolymer_SSW,
            nonbonded_force_kwargs={
                'attractionEnergy': 0,  # base attraction energy for all monomers
                'attractionRadius': attraction_radius,
                'interactionMatrix': interactionMatrix,
                'monomerTypes': isPc,  # the updated colors 
                'extraHardParticlesIdxs': []
            },
            bond_force_kwargs={
                'bondLength': 1,
                'bondWiggleDistance': 0.05
            },
            angle_force_kwargs={
                'k': 1.5
            }
        )
    )
    # ------------ initializing milker; adding bonds ---------
    # copied from addBond
    kbond = a.kbondScalingFactor / (smcBondWiggleDist ** 2)
    bondDist = smcBondDist * a.length_scale
    activeParams = {"length":bondDist,"k":kbond}
    inactiveParams = {"length":bondDist, "k":0}
    milker.setParams(activeParams, inactiveParams)  
    # this step actually puts all bonds in and sets first bonds to be what they should be
    milker.setup(bondForce=a.force_dict['harmonic_bonds'],
                blocks=restartSimulationEveryBlocks)

    # Start simulation without local energy minimization 
    a._apply_forces()
        
    for i in range(restartSimulationEveryBlocks):        
        if i % saveEveryBlocks == (saveEveryBlocks - 1):  
            a.do_block(steps=steps)
        else:
            a.integrator.step(steps)  # do steps without getting the positions from the GPU (faster)
        if i < restartSimulationEveryBlocks - 1: 
            curBonds, pastBonds = milker.step(a.context)  # this updates bonds. You can do something with bonds here
    data = a.get_data()  # save data and step, and delete the simulation
    del a    
    reporter.blocks_only = True  # Write output hdf5-files only for blocks
    time.sleep(0.2)  # wait 200ms for sanity (to let garbage collector do its magic)
    reporter.dump_data()
    
    saveColors = saveFolder + 'colorStates.csv'
    pd.DataFrame(colorStates).to_csv(saveColors)

INFO:root:Performing local energy minimization
INFO:root:adding force harmonic_bonds 0
INFO:root:adding force angle 1
INFO:root:Using periodic boundary conditions
INFO:root:adding force heteropolymer_SSW 2


Exclude neighbouring chain particles from heteropolymer_SSW
Number of exceptions: 599


INFO:root:Particles loaded. Potential energy is 1.526705
INFO:root:before minimization eK=1.528080296560174, eP=1.526705131641394, time=0.0 ps
INFO:root:Particles loaded. Potential energy is 0.209513
INFO:root:after minimization eK=1.528080296560174, eP=0.09603377672581878, time=0.0 ps
INFO:root:block    0 pos[1]=[-2.8 4.5 1.4] dr=8.00 t=949.7ps kin=1.45 pot=1.41 Rg=7.133 SPS=3738 dt=123.2fs dx=33.18pm 
INFO:root:block    1 pos[1]=[6.3 -2.3 4.7] dr=7.97 t=1925.5ps kin=1.46 pot=1.45 Rg=7.201 SPS=4000 dt=119.8fs dx=32.30pm 
INFO:root:block    2 pos[1]=[8.6 3.3 -5.4] dr=8.16 t=2891.8ps kin=1.51 pot=1.45 Rg=7.213 SPS=4278 dt=119.9fs dx=32.89pm 
INFO:root:block    3 pos[1]=[4.7 6.3 1.8] dr=7.26 t=3862.0ps kin=1.55 pot=1.41 Rg=7.303 SPS=4210 dt=121.5fs dx=33.78pm 
INFO:root:block    4 pos[1]=[2.3 -9.7 -2.7] dr=7.95 t=4832.3ps kin=1.59 pot=1.40 Rg=7.175 SPS=3865 dt=120.6fs dx=33.95pm 
INFO:root:block    5 pos[1]=[4.3 -4.1 -2.2] dr=8.37 t=5796.1ps kin=1.43 pot=1.28 Rg=7.086 SPS=3587 dt=123.9fs

0
creating folder
Exclude neighbouring chain particles from heteropolymer_SSW
Number of exceptions: 599


INFO:root:Particles loaded. Potential energy is 1.437166
INFO:root:block    0 pos[1]=[2.6 -0.7 3.5] dr=7.34 t=969.1ps kin=1.51 pot=1.36 Rg=7.194 SPS=4061 dt=120.8fs dx=33.18pm 
INFO:root:block    1 pos[1]=[4.7 -4.3 -0.8] dr=7.01 t=1940.0ps kin=1.51 pot=1.41 Rg=7.046 SPS=3555 dt=120.8fs dx=33.13pm 
INFO:root:block    2 pos[1]=[0.1 -11.4 -5.4] dr=7.59 t=2910.7ps kin=1.52 pot=1.35 Rg=7.018 SPS=3524 dt=119.3fs dx=32.90pm 
INFO:root:block    3 pos[1]=[-8.0 -7.1 -3.7] dr=8.41 t=3880.7ps kin=1.48 pot=1.42 Rg=7.239 SPS=4061 dt=120.4fs dx=32.73pm 
INFO:root:block    4 pos[1]=[-10.7 -8.9 -9.3] dr=7.21 t=4848.6ps kin=1.46 pot=1.38 Rg=7.074 SPS=3980 dt=123.6fs dx=33.35pm 
INFO:root:block    5 pos[1]=[-0.9 -12.8 -14.0] dr=8.27 t=5826.4ps kin=1.54 pot=1.28 Rg=7.178 SPS=3555 dt=123.2fs dx=34.13pm 
INFO:root:block    6 pos[1]=[-14.2 1.5 1.8] dr=7.96 t=6799.3ps kin=1.40 pot=1.31 Rg=7.077 SPS=3571 dt=122.3fs dx=32.29pm 
INFO:root:block    7 pos[1]=[-12.0 -10.3 1.3] dr=7.26 t=7768.0ps kin=1.52 pot=1.40 R

...

INFO:root:Particles loaded. Potential energy is 1.389395
INFO:root:block    0 pos[1]=[-540.2 -284.9 -395.5] dr=7.50 t=969.2ps kin=1.60 pot=1.38 Rg=7.031 SPS=4188 dt=121.2fs dx=34.27pm 
INFO:root:block    1 pos[1]=[-545.7 -282.8 -392.1] dr=7.50 t=1942.5ps kin=1.54 pot=1.34 Rg=7.162 SPS=3941 dt=122.3fs dx=33.89pm 
INFO:root:block    2 pos[1]=[-543.7 -283.8 -388.2] dr=8.73 t=2916.3ps kin=1.53 pot=1.35 Rg=7.260 SPS=3865 dt=121.6fs dx=33.58pm 
INFO:root:block    3 pos[1]=[-545.5 -282.8 -401.8] dr=6.74 t=3897.6ps kin=1.53 pot=1.39 Rg=7.281 SPS=3524 dt=121.1fs dx=33.48pm 
INFO:root:block    4 pos[1]=[-536.1 -270.3 -395.0] dr=8.24 t=4865.8ps kin=1.52 pot=1.35 Rg=7.427 SPS=3306 dt=120.9fs dx=33.34pm 
INFO:root:block    5 pos[1]=[-545.8 -265.6 -396.7] dr=7.16 t=5839.1ps kin=1.50 pot=1.36 Rg=7.278 SPS=4145 dt=122.5fs dx=33.46pm 
INFO:root:block    6 pos[1]=[-540.8 -264.6 -390.1] dr=7.26 t=6808.9ps kin=1.42 pot=1.35 Rg=7.078 SPS=3653 dt=120.8fs dx=32.18pm 
INFO:root:block    7 pos[1]=[-552.0 -282.

1998
creating folder
Exclude neighbouring chain particles from heteropolymer_SSW
Number of exceptions: 599


INFO:root:Particles loaded. Potential energy is 1.311467
INFO:root:block    0 pos[1]=[-544.7 -281.6 -396.6] dr=7.63 t=971.8ps kin=1.42 pot=1.38 Rg=7.068 SPS=3540 dt=121.5fs dx=32.31pm 
INFO:root:block    1 pos[1]=[-555.1 -276.3 -395.7] dr=7.72 t=1943.0ps kin=1.45 pot=1.36 Rg=7.187 SPS=3960 dt=121.9fs dx=32.75pm 
INFO:root:block    2 pos[1]=[-552.1 -287.6 -389.9] dr=7.28 t=2910.0ps kin=1.54 pot=1.42 Rg=7.262 SPS=3524 dt=120.7fs dx=33.45pm 
INFO:root:block    3 pos[1]=[-546.5 -291.6 -396.7] dr=7.98 t=3880.9ps kin=1.54 pot=1.35 Rg=7.179 SPS=3809 dt=121.8fs dx=33.77pm 
INFO:root:block    4 pos[1]=[-546.9 -292.7 -397.2] dr=7.01 t=4850.3ps kin=1.48 pot=1.32 Rg=7.240 SPS=4061 dt=122.9fs dx=33.34pm 
INFO:root:block    5 pos[1]=[-541.6 -290.4 -391.5] dr=7.34 t=5819.6ps kin=1.51 pot=1.41 Rg=7.078 SPS=3390 dt=119.7fs dx=32.85pm 
INFO:root:block    6 pos[1]=[-543.4 -302.5 -383.0] dr=7.17 t=6783.0ps kin=1.50 pot=1.32 Rg=7.070 SPS=3756 dt=122.7fs dx=33.57pm 
INFO:root:block    7 pos[1]=[-539.6 -300.

1999
creating folder
Exclude neighbouring chain particles from heteropolymer_SSW
Number of exceptions: 599


INFO:root:Particles loaded. Potential energy is 1.393235
INFO:root:block    0 pos[1]=[-542.6 -285.5 -379.1] dr=8.19 t=969.8ps kin=1.49 pot=1.36 Rg=7.060 SPS=4301 dt=121.1fs dx=32.98pm 
INFO:root:block    1 pos[1]=[-532.0 -292.3 -377.3] dr=7.20 t=1942.2ps kin=1.47 pot=1.37 Rg=7.319 SPS=4001 dt=120.4fs dx=32.62pm 
INFO:root:block    2 pos[1]=[-536.0 -291.1 -380.5] dr=7.51 t=2911.8ps kin=1.48 pot=1.28 Rg=7.146 SPS=3555 dt=120.3fs dx=32.71pm 
INFO:root:block    3 pos[1]=[-535.2 -297.4 -374.7] dr=7.24 t=3879.6ps kin=1.47 pot=1.39 Rg=7.299 SPS=3865 dt=120.2fs dx=32.51pm 
INFO:root:block    4 pos[1]=[-545.5 -296.5 -369.2] dr=7.74 t=4848.6ps kin=1.62 pot=1.41 Rg=7.119 SPS=3883 dt=119.8fs dx=34.08pm 
INFO:root:block    5 pos[1]=[-539.5 -283.9 -363.2] dr=7.17 t=5822.1ps kin=1.57 pot=1.33 Rg=7.168 SPS=3361 dt=120.7fs dx=33.79pm 
INFO:root:block    6 pos[1]=[-534.4 -295.7 -365.4] dr=7.15 t=6791.0ps kin=1.47 pot=1.33 Rg=7.124 SPS=4188 dt=120.4fs dx=32.58pm 
INFO:root:block    7 pos[1]=[-539.3 -300.

In [7]:
saveColors = saveFolder + 'colorStates.csv'
pd.DataFrame(colorStates).to_csv(saveColors)
colorStates


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
# save parameters as pickle file in folder with the data
import pickle

parList = [density,N1,M,LIFETIME,SEPARATION,iters,totPc,onRate,offRate,contactRadius,attraction_radius,num_chains,MDstepsPerCohesinStep,smcBondWiggleDist,smcBondDist,saveEveryBlocks,restartSimulationEveryBlocks,ctcfSites,ctcfDir,ctcfCapture,ctcfRelease,oneChainMonomerTypes,interactionMatrix,loadProb,nStates,startLevel]
parFile = saveFolder +'pars.pkl'
# Saving the objects:
with open(parFile, 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(parList, f)